In [1]:
import sys

if sys.version_info[0] < 3:
    raise Exception('Python 3 or a more recent version is required.')

#from primitive_pb2 import can_set_my_account_detail
from irohalib import Iroha, IrohaGrpc
from irohalib import IrohaCrypto
import binascii

In [8]:
ADDRESS = 'localhost'
admin_private_key = open('admin@test.priv').read()
user_private_key = IrohaCrypto.private_key()
user_public_key = IrohaCrypto.derive_public_key(user_private_key)
iroha = Iroha('admin@test')

In [3]:
def send_transaction_and_print_status(transaction, port):
    hex_hash = binascii.hexlify(IrohaCrypto.hash(transaction))
    print('Transaction hash = {}, creator = {}'.format(
        hex_hash, transaction.payload.reduced_payload.creator_account_id))
    net = IrohaGrpc(ADDRESS + ':' + str(port))
    net.send_tx(transaction)
    for status in net.tx_status_stream(transaction):
        print(status)

In [4]:
def create_domain_and_asset():
    """
    Creates domain 'domain' and asset 'coin#domain' with precision 2
    """
    commands = [
        iroha.command('CreateDomain', domain_id='domain', default_role='user'),
        iroha.command('CreateAsset', asset_name='coin',
                      domain_id='domain', precision=2)
    ]
    tx = IrohaCrypto.sign_transaction(
        iroha.transaction(commands), admin_private_key)
    send_transaction_and_print_status(tx, 50051)


In [5]:
def add_coin_to_admin():
    """
    Add 1000.00 units of 'coin#domain' to 'admin@test'
    """
    tx = iroha.transaction([
        iroha.command('AddAssetQuantity',
                      asset_id='coin#domain', amount='1000.00')
    ])
    IrohaCrypto.sign_transaction(tx, admin_private_key)
    send_transaction_and_print_status(tx, 50051)


In [9]:
create_domain_and_asset()

Transaction hash = b'7e65ddf7993ca0cc36b5de3a09d9c39f9e2d18c96a8da766e220b01491fe69dc', creator = admin@test
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_FAILED', 2, 2)
('REJECTED', 4, 0)


In [ ]:
def get_account_assets():
    """
    List all the assets of userone@domain
    """
    query = iroha.query('GetAccountAssets', account_id='admin@test')
    IrohaCrypto.sign_query(query, admin_private_key)
    net = IrohaGrpc(ADDRESS + ':' + str(50051))
    response = net.send_query(query)
    data = response.account_assets_response.account_assets
    for asset in data:
        print('Asset id = {}, balance = {}'.format(
            asset.asset_id, asset.balance))

In [ ]:
get_account_assets()

In [55]:
#!/usr/bin/env python3
#
# Copyright Soramitsu Co., Ltd. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0
#

import sys

if sys.version_info[0] < 3:
    raise Exception('Python 3 or a more recent version is required.')

from iroha.primitive_pb2 import can_set_my_account_detail
from iroha import Iroha, IrohaGrpc
from iroha import IrohaCrypto
import binascii

admin_private_key = 'f101537e319568c765b2cc89698325604991dca57b9716b58016b253506cab70'
user_private_key = IrohaCrypto.private_key()
user_public_key = IrohaCrypto.derive_public_key(user_private_key)
iroha = Iroha('admin@test')
net = IrohaGrpc('127.0.0.1:50051')


def trace(func):
    """
    A decorator for tracing methods' begin/end execution points
    """

    def tracer(*args, **kwargs):
        name = func.__name__
        print('\tEntering "{}"'.format(name))
        result = func(*args, **kwargs)
        print('\tLeaving "{}"'.format(name))
        return result

    return tracer


@trace
def send_transaction_and_print_status(transaction):
    hex_hash = binascii.hexlify(IrohaCrypto.hash(transaction))
    print('Transaction hash = {}, creator = {}'.format(
        hex_hash, transaction.payload.reduced_payload.creator_account_id))
    net.send_tx(transaction)
    for status in net.tx_status_stream(transaction):
        print(status)


@trace
def create_domain_and_asset():
    """
    Creates domain 'domain' and asset 'coin#domain' with precision 2
    """
    commands = [
        iroha.command('CreateDomain', domain_id='domain', default_role='user'),
        iroha.command('CreateAsset', asset_name='coin',
                      domain_id='domain', precision=2)
    ]
    tx = IrohaCrypto.sign_transaction(
        iroha.transaction(commands), admin_private_key)
    send_transaction_and_print_status(tx)


@trace
def add_coin_to_admin():
    """
    Add 1000.00 units of 'coin#domain' to 'admin@test'
    """
    tx = iroha.transaction([
        iroha.command('AddAssetQuantity',
                      asset_id='coin#domain', amount='1000.00')
    ])
    IrohaCrypto.sign_transaction(tx, admin_private_key)
    send_transaction_and_print_status(tx)


@trace
def create_account_userone():
    """
    Create account 'userone@domain'
    """
    tx = iroha.transaction([
        iroha.command('CreateAccount', account_name='usertwo', domain_id='domain',
                      public_key=user_public_key)
    ])
    IrohaCrypto.sign_transaction(tx, admin_private_key)
    send_transaction_and_print_status(tx)


@trace
def transfer_coin_from_admin_to_userone():
    """
    Transfer 2.00 'coin#domain' from 'admin@test' to 'userone@domain'
    """
    tx = iroha.transaction([
        iroha.command('TransferAsset', src_account_id='admin@test', dest_account_id='userone@domain',
                      asset_id='coin#domain', description='init top up', amount='2.00')
    ])
    IrohaCrypto.sign_transaction(tx, admin_private_key)
    send_transaction_and_print_status(tx)


@trace
def userone_grants_to_admin_set_account_detail_permission():
    """
    Make admin@test able to set detail to userone@domain
    """
    tx = iroha.transaction([
        iroha.command('GrantPermission', account_id='admin@test',
                      permission=can_set_my_account_detail)
    ], creator_account='userone@domain')
    IrohaCrypto.sign_transaction(tx, user_private_key)
    send_transaction_and_print_status(tx)


@trace
def set_age_to_userone():
    """
    Set age to userone@domain by admin@test
    """
    tx = iroha.transaction([
        iroha.command('SetAccountDetail',
                      account_id='userone@domain', key='age', value='18')
    ])
    IrohaCrypto.sign_transaction(tx, admin_private_key)
    send_transaction_and_print_status(tx)


@trace
def get_coin_info():
    """
    Get asset info for coin#domain
    :return:
    """
    query = iroha.query('GetAssetInfo', asset_id='coin#domain')
    IrohaCrypto.sign_query(query, admin_private_key)

    response = net.send_query(query)
    data = response.asset_response.asset
    print('Asset id = {}, precision = {}'.format(data.asset_id, data.precision))


@trace
def get_account_assets():
    """
    List all the assets of userone@domain
    """
    query = iroha.query('GetAccountAssets', account_id='fabric@domain')
    IrohaCrypto.sign_query(query, admin_private_key)

    response = net.send_query(query)
    data = response.account_assets_response.account_assets
    for asset in data:
        print('Asset id = {}, balance = {}'.format(
            asset.asset_id, asset.balance))


@trace
def get_userone_details():
    """
    Get all the kv-storage entries for userone@domain
    """
    query = iroha.query('GetAccountDetail', account_id='userone@domain')
    IrohaCrypto.sign_query(query, admin_private_key)

    response = net.send_query(query)
    data = response.account_detail_response
    print('Account id = {}, details = {}'.format('userone@domain', data.detail))

In [52]:
user_private_key = IrohaCrypto.private_key()
user_public_key = IrohaCrypto.derive_public_key(user_private_key)

In [49]:
create_domain_and_asset()

	Entering "create_domain_and_asset"
	Entering "send_transaction_and_print_status"
Transaction hash = b'c3666fc526f4f1a17f3deb701dfc5c46fb57e9579ab56dc03d8809751cfaa4e4', creator = admin@test
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_FAILED', 2, 3)
('REJECTED', 4, 0)
	Leaving "send_transaction_and_print_status"
	Leaving "create_domain_and_asset"


In [53]:
user_public_key

b'85f0ebe85f343c183d44f00024a14e46031b591d35ea0d8de852796d143f8713'

In [50]:
add_coin_to_admin()

	Entering "add_coin_to_admin"
	Entering "send_transaction_and_print_status"
Transaction hash = b'afac8c17f408c779cc12be58bef790ed6879d88b56a754b120fde67cf8cc8ea4', creator = admin@test
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "send_transaction_and_print_status"
	Leaving "add_coin_to_admin"


In [54]:
create_account_userone()

	Entering "create_account_userone"
	Entering "send_transaction_and_print_status"
Transaction hash = b'beff2e67324614ccd62d57e79c8fec7bbc47e018c567ff93fa4848bf25b9d931', creator = admin@test
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_FAILED', 2, 4)
('REJECTED', 4, 0)
	Leaving "send_transaction_and_print_status"
	Leaving "create_account_userone"


In [30]:
transfer_coin_from_admin_to_userone()

	Entering "transfer_coin_from_admin_to_userone"
	Entering "send_transaction_and_print_status"
Transaction hash = b'2a351a8fbdaf1db230e0a4755590f0d3b4a1cd537bbd5f5a6a963f55216c4dce', creator = admin@test
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "send_transaction_and_print_status"
	Leaving "transfer_coin_from_admin_to_userone"


In [15]:
userone_grants_to_admin_set_account_detail_permission()

	Entering "userone_grants_to_admin_set_account_detail_permission"
	Entering "send_transaction_and_print_status"
Transaction hash = b'1411d3d83e736e692a4e05c35d589809d82c34c01763fed22306421f7ff014fa', creator = userone@domain
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "send_transaction_and_print_status"
	Leaving "userone_grants_to_admin_set_account_detail_permission"


In [ ]:
set_age_to_userone()
get_coin_info()

In [56]:
get_account_assets()

	Entering "get_account_assets"
	Leaving "get_account_assets"


In [ ]:
get_userone_details()

print('done')